# Combining the two sets of data logged from the SSN website 

Having previously cleaned the datasets ([1](./ExamineANMReadings.ipynb), [2](./ExamineElectricityReadings.ipynb)), such that they have a maximum of one record in any minute period, they now need to be combined into a single dataset such that for each data point, there exists a complete record of electricity readings and 'ANM' status.

This process is complicated due to the ANM records having been stored only when there is a change, as opposed to every reading.  This was done to reduce the overall number of records stored and consequently, minimise storage requirements. There is no lost data as the duration of any particular event can be inferred from the start time of the subsequent one.   

Two methods are examined here:
  * combining the tables into a single database to run a single SQL query
  * using a python script to run through each entry in the readings table and pairing it with the appropriate entry in the ANM log

A script has also been written to parse the JSON format logged data and combine it into a single database.
the algorithm performs a sort on the ANM log (to identify the most recent relevant record) for each row of the readings database, which is very inefficient. 

Consequently, the processing time for the [final merge and write](#Final-write) is significant.  
***Processing the merge and write cell in this document took nearly an hour on my system.***


However, the resulting database is satisfactory and will be used to proceed with the analysis. 

---

## Initialise working environment

* load iPython  %sql magic extension  
* connect to databases and check they are as expected

In [1]:
%load_ext sql

### Electricity log database
---

In [61]:
%sql sqlite:///./database/eleclog_copy.db

'Connected: @./database/eleclog_copy.db'

In [63]:
%sql @./database/eleclog_copy.db SELECT COUNT(*) FROM readings;

Done.


COUNT(*)
395525


In [64]:
%sql @./database/eleclog_copy.db SELECT sql FROM sqlite_master WHERE name = 'readings';

Done.


sql
"CREATE TABLE readings (timestamp DATETIME, demand NUMERIC, demand_max NUMERIC, anm_generated NUMERIC, non_anm_generated NUMERIC, total_generation_capacity NUMERIC)"


In [65]:
#copy readings table to a local variable
readings_orig = %sql @./database/eleclog_copy.db select * from readings order by timestamp;

Done.


###  ANM event database
---

In [6]:
%sql sqlite:///./database/ANMlog_copy.db

'Connected: @./database/ANMlog_copy.db'

In [66]:
%sql @./database/ANMlog_copy.db SELECT COUNT(*) FROM records;

Done.


COUNT(*)
15711


In [67]:
%sql @./database/ANMlog_copy.db SELECT sql FROM sqlite_master WHERE name = 'records';

Done.


sql
"CREATE TABLE records(timestamp DATETIME, log TEXT)"


In [68]:
#copy readings table to a local variable
records = %sql @./database/ANMlog_copy.db select * from records order by timestamp;

Done.


###  Create combined database
---

In [10]:
%sql sqlite:///./database/combined.db

'Connected: @./database/combined.db'

In [11]:
print(readings_orig[1], records[1])

('2019-01-16 14:52:03', 23.48, 35.7, 14.107, 17.92, 57.1) ('2019-01-16 22:46:25', '{"1": ["ok", "ok", "ok"], "1A": ["ok", "ok", "ok"], "2": ["warning", "ok", "ok"], "2A": ["remove", "ok", "ok"], "2B": ["warning", "ok", "ok"], "3": ["ok", "ok", "ok"], "4": ["ok", "ok", "ok"], "4A": ["ok", "ok", "ok"], "Core": ["ok", "ok", "ok"]}')


%sql magic provides for creation of tables directly from DataFrame types, using the 'persists' command

It is unecessary to create the tables in the database prior to loading the data
>
```python
%sql CREATE TABLE readings (timestamp DATETIME, demand NUMERIC, demand_max NUMERIC, anm_generated NUMERIC, non_anm_generated NUMERIC, total_generation_capacity NUMERIC);
%sql CREATE TABLE records(timestamp DATETIME, log TEXT);
```

*If the table already exists in the external database file the 'persist' function fails.  To speed up development, these commands have been commented out in the following cells.* 

In [12]:
records.DataFrame().head()

,timestamp,log
0,2019-01-16 22:42:33,"{""2"": [""warning"", ""ok"", ""ok""], ""2B"": [""warning..."
1,2019-01-16 22:46:25,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""..."
2,2019-01-16 22:59:22,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""..."
3,2019-01-16 23:03:05,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""..."
4,2019-01-16 23:06:26,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""..."


In [13]:
# -persist functions have been commented out as they are only needed to create the database file './database/combined.db'. (i.e. the first time this page is run) 

records = records.DataFrame()
# %sql @./database/combined.db persist records 

readings = readings_orig.DataFrame()
# %sql @./database/combined.db persist readings

---

## Joins

### Simple join matching records from ANM logs with electricity readings.

Using a substring of the timestamp matches records by the 'minute' avoiding incongruity with differing 'seconds'

However, it only matches each record from the ANM log with one from the electricity readings.  Hence a large number of electricity readings are omitted.

It is also noted that the first match is for row 470 of elec_readings with row 0 of ANM_logs. Thus the first 469 rows are omitted from the final [join](#Final-write). 

In [69]:
%%sql @./database/combined.db
select
    *
from
    readings
left join
    records
where
    substr(readings.timestamp, 0, 17) == substr(records.timestamp, 0, 17)
limit
    5
    ;

Done.


index,timestamp,demand,demand_max,anm_generated,non_anm_generated,total_generation_capacity,index_1,timestamp_1,log
470,2019-01-16 22:42:03,19.66,35.7,17.145,17.075,57.099999999999994,0,2019-01-16 22:42:33,"{""2"": [""warning"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""], ""1"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""]}"
474,2019-01-16 22:46:03,18.7,35.7,14.572,16.457,57.1,1,2019-01-16 22:46:25,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""warning"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
487,2019-01-16 22:59:03,18.28,35.7,15.618,16.093,57.099999999999994,2,2019-01-16 22:59:22,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""ok"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
491,2019-01-16 23:03:05,18.56,35.7,13.922,18.143,57.1,3,2019-01-16 23:03:05,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""ok"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
494,2019-01-16 23:06:03,18.2,35.7,16.457,17.996,57.099999999999994,4,2019-01-16 23:06:26,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""warning"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"


Extract from AMN log the most recent two events for a particular time.

In [101]:
%%sql @./database/combined.db
select * from records where timestamp < '2019-01-17 09:35:11' order by timestamp desc limit 2;

Done.


index,timestamp,log
12,2019-01-17 09:34:10,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""warning"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
11,2019-01-17 09:31:11,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""remove"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"


In [71]:
# match most recent ANM log event to a number of sample readings from  electricity log

for reading in readings_orig[30000:30010]:
    readingdate= "'" + reading['timestamp'] + "'"
    anmstatus = %sql @./database/combined.db select * from records where timestamp < $readingdate order by timestamp desc limit 1;
    print('electricity reading timestamp: ',reading['timestamp'],' | ANM event timestamp: ', anmstatus[0]['timestamp'])

Done.
electricity reading timestamp:  2019-02-06 18:14:09  | ANM event timestamp:  2019-02-06 18:13:09
Done.
electricity reading timestamp:  2019-02-06 18:15:05  | ANM event timestamp:  2019-02-06 18:14:13
Done.
electricity reading timestamp:  2019-02-06 18:16:05  | ANM event timestamp:  2019-02-06 18:14:13
Done.
electricity reading timestamp:  2019-02-06 18:17:05  | ANM event timestamp:  2019-02-06 18:14:13
Done.
electricity reading timestamp:  2019-02-06 18:18:05  | ANM event timestamp:  2019-02-06 18:14:13
Done.
electricity reading timestamp:  2019-02-06 18:19:05  | ANM event timestamp:  2019-02-06 18:18:10
Done.
electricity reading timestamp:  2019-02-06 18:20:05  | ANM event timestamp:  2019-02-06 18:18:10
Done.
electricity reading timestamp:  2019-02-06 18:21:04  | ANM event timestamp:  2019-02-06 18:20:09
Done.
electricity reading timestamp:  2019-02-06 18:22:05  | ANM event timestamp:  2019-02-06 18:20:09
Done.
electricity reading timestamp:  2019-02-06 18:23:05  | ANM event ti

#### Comment

Although this appears to have worked as expected, notice that the first and second results indicate something amiss.

* In the first result(30000), the electricity reading was taken on '2019-02-06 18:14:09'  whilst the matched previous ANM event occurred on '2019-02-06 18:13:09'
* In the Second result (30001), the ANM reading was taken on '2019-02-06 18:14:13' - only a few seconds after electricity reading 30000 was taken.

The data logger starts the logging processes simultaneously with a view to obtaining concurrent readings and events from the live data feed.  However, the timestamp on each record reflects the time the record was written to the database file.  This indicates that retrieving the ANM log takes longer than retrieving an electricity reading. The result is the mismatch of records shown above where, clearly, the electricity reading should be matched to the ANM log that occurs a few seconds later.

This can be achieved by perfoming the search using truncated timestamps - ommitting the 'seconds' field.

In [72]:
# match most relevant ANM log event to a reading from the electricity log, where the ANM event is either the most recent or occurs within the same minute period.
# this is done by performing the search truncating the timestamps to exclude 'seconds' 

reading = readings_orig[30000]
readingdate= "'" + reading['timestamp'] + "'"
anmstatus = %sql @./database/combined.db select * from records where substr(timestamp, 0, 17) <= substr($readingdate, 0, 17) order by timestamp desc limit 1;
print(reading['timestamp'], anmstatus[0]['timestamp'])

Done.
2019-02-06 18:14:09 2019-02-06 18:14:13


In [73]:
import json

In [58]:
for reading in readings_orig[30000:30010]:
    readingdate= "'"+reading['timestamp']+"'"
    anmrecord = %sql @./database/combined.db select * from records where substr(timestamp, 0, 17) <= substr($readingdate, 0, 17) order by timestamp desc limit 1;
    anmstatus = json.loads(anmrecord[0]['log'])
    print(reading['timestamp'],
          reading['demand'],
          reading['demand_max'],
          reading['anm_generated'],
          reading['non_anm_generated'],
          reading['total_generation_capacity'],
          anmrecord[0]['timestamp'],
          anmstatus['Core'][0],
          anmstatus['Core'][1],
          anmstatus['Core'][2],
          anmstatus['1'][0],
          anmstatus['1'][1],
          anmstatus['1'][2],
          anmstatus['2'][0],
          anmstatus['2'][1],
          anmstatus['2'][2],
          anmstatus['2A'][0],
          anmstatus['2A'][1],
          anmstatus['2A'][2],
          anmstatus['2B'][0],
          anmstatus['2B'][1],
          anmstatus['2B'][2],
          anmstatus['3'][0],
          anmstatus['3'][1],
          anmstatus['3'][2],
          anmstatus['4'][0],
          anmstatus['4'][1],
          anmstatus['4'][2],
          anmstatus['4A'][0],
          anmstatus['4A'][1],
          anmstatus['4A'][2])
    
    

Done.
2019-02-06 18:14:09 21.62 35.7 7.592 14.065 57.099999999999994 2019-02-06 18:14:13 ok ok ok ok ok ok warning ok ok warning ok ok ok ok ok ok ok ok ok ok ok ok ok ok
Done.
2019-02-06 18:15:05 21.88 35.7 10.149 14.043 57.1 2019-02-06 18:14:13 ok ok ok ok ok ok warning ok ok warning ok ok ok ok ok ok ok ok ok ok ok ok ok ok
Done.
2019-02-06 18:16:05 21.59 35.7 9.881 12.069 57.1 2019-02-06 18:14:13 ok ok ok ok ok ok warning ok ok warning ok ok ok ok ok ok ok ok ok ok ok ok ok ok
Done.
2019-02-06 18:17:05 21.67 35.7 9.849 12.467 57.099999999999994 2019-02-06 18:14:13 ok ok ok ok ok ok warning ok ok warning ok ok ok ok ok ok ok ok ok ok ok ok ok ok
Done.
2019-02-06 18:18:05 21.53 35.7 8.921 12.43 57.1 2019-02-06 18:18:10 ok ok ok ok ok ok ok ok ok warning ok ok ok ok ok ok ok ok ok ok ok ok ok ok
Done.
2019-02-06 18:19:05 21.41 35.7 9.002 12.262 57.1 2019-02-06 18:18:10 ok ok ok ok ok ok ok ok ok warning ok ok ok ok ok ok ok ok ok ok ok ok ok ok
Done.
2019-02-06 18:20:05 21.65 35.7 8.9

# Create final database file

In [75]:
%sql sqlite:///./database/ssen_merged.db
%sql sqlite:///./database/combined.db   

'Connected: @./database/combined.db'

In [99]:
%sql @./database/combined.db drop table merged_records

(sqlite3.OperationalError) no such table: merged_records
[SQL: drop table merged_records;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [103]:
%sql @./database/ssen_merged.db drop table merged_records

Done.


[]

In [104]:
%%sql @./database/ssen_merged.db
CREATE TABLE merged_records (
    elec_timestamp DATETIME,
    demand NUMERIC,
    demand_max NUMERIC,
    anm_generated NUMERIC,
    non_anm_generated NUMERIC,
    total_generation_capacity NUMERIC,
    ANM_timestamp DATETIME,
    operation_core TEXT NOT NULL,
    eqpt_core TEXT NOT NULL,
    site_core TEXT NOT NULL,
    operation_1 TEXT NOT NULL,
    eqpt_1 TEXT NOT NULL,
    site_1 TEXT NOT NULL,
    operation_1A TEXT NOT NULL,
    eqpt_1A TEXT NOT NULL,
    site_1A TEXT NOT NULL,
    operation_2 TEXT NOT NULL,
    eqpt_2 TEXT NOT NULL,
    site_2 TEXT NOT NULL,
    operation_2A TEXT NOT NULL,
    eqpt_2A TEXT NOT NULL,
    site_2A TEXT NOT NULL,
    operation_2B TEXT NOT NULL,
    eqpt_2B TEXT NOT NULL,
    site_2B TEXT NOT NULL,
    operation_3 TEXT NOT NULL,
    eqpt_3 TEXT NOT NULL,
    site_3 TEXT NOT NULL,
    operation_4 TEXT NOT NULL,
    eqpt_4 TEXT NOT NULL,
    site_4 TEXT NOT NULL,
    operation_4A TEXT NOT NULL,
    eqpt_4A TEXT NOT NULL,
    site_4A TEXT NOT NULL
    )
;

Done.


[]

In [109]:

reading = readings_orig[30000]
readingdate= "'"+reading['timestamp']+"'"
anmrecord = %sql @./database/combined.db select * from records where substr(timestamp, 0, 17) <= substr($readingdate, 0, 17) order by timestamp desc limit 1;
anmstatus = json.loads(anmrecord[0]['log'])
insert_query = f'''
insert into
    records values (
    '{reading['timestamp']}',
          {reading['demand']},
          {reading['demand_max']},
          {reading['anm_generated']},
          {reading['non_anm_generated']},
          {reading['total_generation_capacity']},
          '{anmrecord[0]['timestamp']}',
          '{anmstatus['Core'][0]}',
          '{anmstatus['Core'][1]}',
          '{anmstatus['Core'][2]}',
          '{anmstatus['1'][0]}',
          '{anmstatus['1'][1]}',
          '{anmstatus['1'][2]}',
          '{anmstatus['1A'][0]}',
          '{anmstatus['1A'][1]}',
          '{anmstatus['1A'][2]}',
          '{anmstatus['2'][0]}',
          '{anmstatus['2'][1]}',
          '{anmstatus['2'][2]}',
          '{anmstatus['2A'][0]}',
          '{anmstatus['2A'][1]}',
          '{anmstatus['2A'][2]}',
          '{anmstatus['2B'][0]}',
          '{anmstatus['2B'][1]}',
          '{anmstatus['2B'][2]}',
          '{anmstatus['3'][0]}',
          '{anmstatus['3'][1]}',
          '{anmstatus['3'][2]}',
          '{anmstatus['4'][0]}',
          '{anmstatus['4'][1]}',
          '{anmstatus['4'][2]}',
          '{anmstatus['4A'][0]}',
          '{anmstatus['4A'][1]}',
          '{anmstatus['4A'][2]}'
          )'''
print(insert_query) 

Done.

insert into
    records values (
    '2019-02-06 18:14:09',
          21.62,
          35.7,
          7.592,
          14.065,
          57.099999999999994,
          '2019-02-06 18:14:13',
          'ok',
          'ok',
          'ok',
          'ok',
          'ok',
          'ok',
          'ok',
          'ok',
          'ok',
          'warning',
          'ok',
          'ok',
          'warning',
          'ok',
          'ok',
          'ok',
          'ok',
          'ok',
          'ok',
          'ok',
          'ok',
          'ok',
          'ok',
          'ok',
          'ok',
          'ok',
          'ok'
          )


### Final write

This cell performs the final merge of the two logged databases - electricity readings and ANM logs.

The command `%%capture` suppresses output to the notebook: otherwise each database transaction produces a `Done.` message into this notebook.

The command is run from entry '470' as there are no ANM log records prior to this timestamp ([see above](#Simple-join-matching-records-from-ANM-logs-with-electricity-readings.))

With a sort peformed on the anm log (to identify the most recent relevant record) for each row of the readings database, the algorithm is very inefficient.  
***Processing the whole database took over half an hour on my system.***  
However, the resulting database is satisfactory and will be used to proceed with the analysis. 

In [105]:
%%capture
for reading in readings_orig[470:]:
    readingdate= "'"+reading['timestamp']+"'"
    anmrecord = %sql @./database/combined.db select * from records where substr(timestamp, 0, 17) <= substr($readingdate, 0, 17) order by timestamp desc limit 1;
    anmstatus = json.loads(anmrecord[0]['log'])
    insert_query = f'''
    insert into
    merged_records values (
    '{reading['timestamp']}',
          {reading['demand']},
          {reading['demand_max']},
          {reading['anm_generated']},
          {reading['non_anm_generated']},
          {reading['total_generation_capacity']},
          '{anmrecord[0]['timestamp']}',
          '{anmstatus['Core'][0]}',
          '{anmstatus['Core'][1]}',
          '{anmstatus['Core'][2]}',
          '{anmstatus['1'][0]}',
          '{anmstatus['1'][1]}',
          '{anmstatus['1'][2]}',
          '{anmstatus['1A'][0]}',
          '{anmstatus['1A'][1]}',
          '{anmstatus['1A'][2]}',
          '{anmstatus['2'][0]}',
          '{anmstatus['2'][1]}',
          '{anmstatus['2'][2]}',
          '{anmstatus['2A'][0]}',
          '{anmstatus['2A'][1]}',
          '{anmstatus['2A'][2]}',
          '{anmstatus['2B'][0]}',
          '{anmstatus['2B'][1]}',
          '{anmstatus['2B'][2]}',
          '{anmstatus['3'][0]}',
          '{anmstatus['3'][1]}',
          '{anmstatus['3'][2]}',
          '{anmstatus['4'][0]}',
          '{anmstatus['4'][1]}',
          '{anmstatus['4'][2]}',
          '{anmstatus['4A'][0]}',
          '{anmstatus['4A'][1]}',
          '{anmstatus['4A'][2]}'
          )'''
    %sql @./database/ssen_merged.db $insert_query

### check head and tail of merged database

In [106]:
%sql @./database/ssen_merged.db select * from merged_records limit 10;

Done.


elec_timestamp,demand,demand_max,anm_generated,non_anm_generated,total_generation_capacity,ANM_timestamp,operation_core,eqpt_core,site_core,operation_1,eqpt_1,site_1,operation_1A,eqpt_1A,site_1A,operation_2,eqpt_2,site_2,operation_2A,eqpt_2A,site_2A,operation_2B,eqpt_2B,site_2B,operation_3,eqpt_3,site_3,operation_4,eqpt_4,site_4,operation_4A,eqpt_4A,site_4A
2019-01-16 22:42:03,19.66,35.7,17.145,17.075,57.099999999999994,2019-01-16 22:42:33,ok,ok,ok,ok,ok,ok,ok,ok,ok,warning,ok,ok,remove,ok,ok,warning,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok
2019-01-16 22:43:03,19.02,35.7,16.829,17.52,57.099999999999994,2019-01-16 22:42:33,ok,ok,ok,ok,ok,ok,ok,ok,ok,warning,ok,ok,remove,ok,ok,warning,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok
2019-01-16 22:44:03,18.4,35.7,14.394,17.245,57.099999999999994,2019-01-16 22:42:33,ok,ok,ok,ok,ok,ok,ok,ok,ok,warning,ok,ok,remove,ok,ok,warning,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok
2019-01-16 22:45:03,18.25,35.7,13.674,17.446,57.099999999999994,2019-01-16 22:42:33,ok,ok,ok,ok,ok,ok,ok,ok,ok,warning,ok,ok,remove,ok,ok,warning,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok
2019-01-16 22:46:03,18.7,35.7,14.572,16.457,57.1,2019-01-16 22:46:25,ok,ok,ok,ok,ok,ok,ok,ok,ok,warning,ok,ok,remove,ok,ok,warning,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok
2019-01-16 22:47:03,18.57,35.7,14,16.448,57.1,2019-01-16 22:46:25,ok,ok,ok,ok,ok,ok,ok,ok,ok,warning,ok,ok,remove,ok,ok,warning,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok
2019-01-16 22:48:03,18.3,35.7,13.504,16.564,57.1,2019-01-16 22:46:25,ok,ok,ok,ok,ok,ok,ok,ok,ok,warning,ok,ok,remove,ok,ok,warning,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok
2019-01-16 22:49:03,18.56,35.7,13.334,16.952,57.099999999999994,2019-01-16 22:46:25,ok,ok,ok,ok,ok,ok,ok,ok,ok,warning,ok,ok,remove,ok,ok,warning,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok
2019-01-16 22:50:03,19.07,35.7,13.701,16.934,57.099999999999994,2019-01-16 22:46:25,ok,ok,ok,ok,ok,ok,ok,ok,ok,warning,ok,ok,remove,ok,ok,warning,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok
2019-01-16 22:51:03,18.7,35.7,15.406,17.383,57.099999999999994,2019-01-16 22:46:25,ok,ok,ok,ok,ok,ok,ok,ok,ok,warning,ok,ok,remove,ok,ok,warning,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok


In [107]:
%sql @./database/ssen_merged.db select * from merged_records order by elec_timestamp desc limit 10;

Done.


elec_timestamp,demand,demand_max,anm_generated,non_anm_generated,total_generation_capacity,ANM_timestamp,operation_core,eqpt_core,site_core,operation_1,eqpt_1,site_1,operation_1A,eqpt_1A,site_1A,operation_2,eqpt_2,site_2,operation_2A,eqpt_2A,site_2A,operation_2B,eqpt_2B,site_2B,operation_3,eqpt_3,site_3,operation_4,eqpt_4,site_4,operation_4A,eqpt_4A,site_4A
2019-10-27 21:48:06,14.06,35.7,8.079,9.354,57.1,2019-10-27 13:35:16,remove,ok,ok,ok,ok,ok,remove,ok,remove,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok
2019-10-27 21:47:05,14.17,35.7,7.515,9.543,57.1,2019-10-27 13:35:16,remove,ok,ok,ok,ok,ok,remove,ok,remove,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok
2019-10-27 21:46:06,14.01,35.7,8.042,9.379,57.1,2019-10-27 13:35:16,remove,ok,ok,ok,ok,ok,remove,ok,remove,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok
2019-10-27 21:45:06,14.21,35.7,8.598998,8.649,57.1,2019-10-27 13:35:16,remove,ok,ok,ok,ok,ok,remove,ok,remove,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok
2019-10-27 21:44:10,14.5,35.7,7.237001,9.06,57.1,2019-10-27 13:35:16,remove,ok,ok,ok,ok,ok,remove,ok,remove,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok
2019-10-27 21:43:06,14.8,35.7,7.92,9.247,57.1,2019-10-27 13:35:16,remove,ok,ok,ok,ok,ok,remove,ok,remove,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok
2019-10-27 21:42:04,14.58,35.7,7.264,8.147,57.1,2019-10-27 13:35:16,remove,ok,ok,ok,ok,ok,remove,ok,remove,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok
2019-10-27 21:41:05,14.93,35.7,7.738,9.438,57.1,2019-10-27 13:35:16,remove,ok,ok,ok,ok,ok,remove,ok,remove,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok
2019-10-27 21:40:04,14.88,35.7,7.727002,11.358,57.099999999999994,2019-10-27 13:35:16,remove,ok,ok,ok,ok,ok,remove,ok,remove,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok
2019-10-27 21:39:06,15.05,35.7,7.792,12.829,57.1,2019-10-27 13:35:16,remove,ok,ok,ok,ok,ok,remove,ok,remove,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok


### write out as .csv file 

In [110]:
merged_data = %sql @./database/ssen_merged.db select * from merged_records order by elec_timestamp;

Done.


In [111]:
merged_data.csv(filename='./database/ssen_merged.csv')

CSV results at C:\Users\Stuart\Documents\College\ssen\dataclean\./database/ssen_merged.csv